In [1]:
# import all needed libraries
import pandas as pd
import os
import requests
from sqlalchemy import create_engine
from dotenv import load_dotenv
from datetime import datetime

# load environment variables
load_dotenv()

try:
    # try to retrieve the values from the environment variables
    # exchange rates API
    erAppId = os.environ["APP_ID"]
    # select neon postgre database or leave both updating in parallel
    databaseUrl = os.environ["DB_URL"]
    new_databaseUrl = os.environ["NEW_DB_URL"]
except KeyError as e:
    # handle the case where an expected environment variable is missing
    raise ValueError(f"Missing environment variable: {e}")

# create a connection to the neon postgre database
engine = create_engine(databaseUrl)
new_engine = create_engine(new_databaseUrl)

# create a dictionary of currencies to fetch 
currency_mapping = {
    "ARS": "ARS",
    "BRL": "BRL",
    "BOB": "BOB",
    "CAD": "CAD",
    "CLF": "CLF",
    "CLP": "CLP",
    "CNY": "CNY",
    "COP": "COP",
    "CRC": "CRC",
    "DOP": "DOP",
    "EUR": "EUR",
    "GBP": "GBP",
    "MXN": "MXN",
    "PEN": "PEN",
    "PYG": "PYG",
    "USD": "USD",
    "UYU": "UYU",
    # add more currencies ( available currencies ---> https://docs.openexchangerates.org/reference/supported-currencies)
}

# construct the API URL ---> DESCARTADAS POR TENER BASE EN EUROS POR DEFAULT, NO SER GRATIS O NO TENER ALGUNAS DE LAS MONDEDAS QUE REQUERIMOS
# url = f'http://api.exchangeratesapi.io/v1/latest?access_key={api_key}&symbols=USD,MXN,BRL,ARS,CLP,COP,PEN'
# url = f'http://data.fixer.io/api/latest?access_key={api_key}&base=USD&symbols=MXN,BRL,ARS,CLP,COP,PEN'
# url = f'https://api.currencybeacon.com/v1/latest?access_key={api_key}&symbols=MXN,BRL,ARS,CLP,COP,PEN'

# construct the API URL & send a GET request to the API
url = f'https://openexchangerates.org/api/latest.json?app_id={erAppId}'
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)

# manage api response
data = response.json()
# print(data)

# extract all exchange rates from the data
rates = data['rates']
# print(exchange_rates)

# map only the used currencies exchage rates based on USD
usd_er = {currency_mapping[key]: rates[key] for key in currency_mapping.keys()}
# print(usd_er)

# create DataFrame from the dictionary
er_df = pd.DataFrame(list(usd_er.items()), columns=["currency", "exchange_rate"])

# add a timestamp column with the current date and time without time zone
er_df['timestamp'] = datetime.now()

# reorder columns for better readability
er_df = er_df[['timestamp', 'currency', 'exchange_rate']].round({'exchange_rate': 2})

# pivot the DataFrame to have currencies as columns
er_df_pivoted = er_df.pivot(index='timestamp', columns='currency', values='exchange_rate').reset_index()

# reorder used currencies columns first for better readability 
er_df_pivoted = er_df_pivoted[['timestamp', 'USD', 'ARS', 'CLP', 'CLF', 'PEN', 'COP', 'MXN', 'BRL', 'EUR'] + [col for col in er_df_pivoted.columns if col not in ['timestamp', 'USD', 'ARS', 'CLP', 'CLF', 'PEN', 'COP', 'MXN', 'BRL', 'EUR']]]

# and finally insert the dataframe into the PostgreSQL table named 'usd_er'
# use 'replace' to reset table or 'append' to accumulate historical exchange rates data
table_name = 'usd_er'
try:
    # er_df_pivoted.to_sql(table_name, con=engine, if_exists='replace', index=False)
    er_df_pivoted.to_sql(table_name, con=engine, if_exists='append', index=False)
finally:
    # close the database connection
    engine.dispose()
try:
    # er_df_pivoted.to_sql(table_name, con=engine, if_exists='replace', index=False)
    er_df_pivoted.to_sql(table_name, con=new_engine, if_exists='append', index=False)
finally:
    # close the database connection
    new_engine.dispose()